<a href="https://colab.research.google.com/github/RodolfoFerro/human-motion-prediction-pytorch/blob/master/notebooks/human_motion_prediction_Custom_Model_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# human-motion-prediction 🕺🏻

> **Note:** This repo is a fork of this one: https://github.com/cimat-ris/human-motion-prediction-pytorch
>
> The code has been refactored preserving the logic and structure, but adding functionalities to run it in Google Colab.

> Pytorch implementation of:
>
> &nbsp;&nbsp; Julieta Martinez, Michael J. Black, Javier Romero. _**On human motion prediction using recurrent neural networks**_. In CVPR 17.
> 
> The paper can be found on arXiv: [https://arxiv.org/pdf/1705.02445.pdf](https://arxiv.org/pdf/1705.02445.pdf)

Find the repo of this code here: https://github.com/RodolfoFerro/human-motion-prediction-pytorch.git

Clone the repository with code:

In [ ]:
!git clone https://github.com/RodolfoFerro/human-motion-prediction-pytorch.git
%cd human-motion-prediction-pytorch
!ls

Dowload the data:

> We need to install `gdown` to download the data from Google Drive into our local folder.

In [ ]:
!mkdir data
%cd data

!gdown https://drive.google.com/uc?id=1hqE6GrWZTBjVzmbehUBO7NTrbEgDNqbH
!unzip -q h3.6m.zip
!rm h3.6m.zip
%cd ..
!ls

### Custom model

You can create a custom model by creating a new object with inheritance of `nn.Module`:

In [ ]:
"""Sequence-to-sequence model for human motion prediction."""

import logging

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn


class MotionGenerator(nn.Module):
    """Sequence-to-sequence model for human motion prediction"""

    def __init__(
            self,
            source_seq_len,
            target_seq_len,
            n_samples,
            rnn_size,  # recurrent layer hidden size
            batch_size,
            learning_rate,
            learning_rate_decay_factor,
            number_of_actions,
            len_z=128,
            dropout=0.3):
        """Constructor of the class.
        
        Parameters
        ----------
        source_seq_len: int
            Length of the input sequence.
        target_seq_len: int
            Length of the target sequence.
        n_samples: int
            Number of trajectories to be generated.
        rnn_size: int
            Number of units in the rnn.
        batch_size: int
            The size of the batches used during training; the model
            construction is independent of batch_size, so it can be
            changed after initialization if this is convenient, e.g.,
            for decoding.
        learning_rate: float
            Learning rate to start with.
        learning_rate_decay_factor: 
            Decay learning rate by this much when needed.
        len_z: int
            Size of random vector.
        number_of_actions: int
            Number of classes we have.
        """

        super(MotionGenerator, self).__init__()

        self.human_dofs = 54
        self.input_size = self.human_dofs + number_of_actions + len_z
        self.len_z = len_z

        logging.info(f'Input size is {self.input_size}')
        self.source_seq_len = source_seq_len
        self.target_seq_len = target_seq_len
        self.rnn_size = rnn_size
        self.batch_size = batch_size
        self.dropout = dropout
        self.n_samples = n_samples

        # Create the RNN that will summarize the state
        self.cell = torch.nn.GRUCell(self.input_size, self.rnn_size)
        self.cell_out = torch.nn.GRUCell(self.input_size - len_z, self.rnn_size)
        self.fc1 = nn.Linear(self.rnn_size, self.input_size - len_z)

    def forward(self, z, encoder_inputs, decoder_inputs, device):
        """Forward pass of the model.

        Parameters
        ----------
        z: np.array
            A random generated input vector used as input.
        encoder_inputs : torch.Tensor
            The input to the encoder.
        decoder_inputs : torch.Tensor
            The input to the decoder.
        device : torch.device
            The device on which to do the computation.
        
        Returns
        -------
        outputs : torch.Tensor
            The transposed output of the model.
        """

        def loop_function(prev, i):
            return prev

        # Adds noise
        inputs_tensor = torch.cat((encoder_inputs, z), dim=2)
        batch_size = inputs_tensor.shape[0]
        
        # To pass these data through a RNN we need to switch the first
        # two dimensions
        encoder_inputs = torch.transpose(inputs_tensor, 0, 1)
        decoder_inputs = torch.transpose(decoder_inputs, 0, 1)
        state = torch.zeros(batch_size, self.rnn_size).to(device)

        # Encoding
        for i in range(self.source_seq_len - 1):
            # Apply the RNN cell
            state = self.cell(encoder_inputs[i], state)

            # Apply dropout in training
            state = F.dropout(state, self.dropout, training=self.training)

        outputs = []
        prev = None

        # Decoding, sequentially
        # TODO: Gen n_samples
        for j in range(self.n_samples):
            outs = []
            for i, inp in enumerate(decoder_inputs):
                # Use teacher forcing?
                if prev is not None:
                    inp = loop_function(prev, i)
                #inp = inp.detach()

                state = self.cell_out(inp, state)

                # Output is seen as a residual to the previous value
                output = inp + self.fc1(
                    F.dropout(state, self.dropout, training=self.training))
                outs.append(output.view([1, batch_size, self.input_size - self.len_z]))
                prev = output
            
            outs = torch.cat(outs, 0)
            
            # Size should be batch_size x target_seq_len x input_size
            outs = torch.transpose(outs, 0, 1)
            
            outputs.append(outs)

        return outputs

    def get_batch(self, data, actions, device):
        """Get a random batch of data from the specified bucket, prepare
        for step.
        
        Parameters
        ----------
        data:
            A list of sequences of size n-by-d to fit the model to.
        actions:
            A list of the actions we are using
        device:
            The device on which to do the computation (cpu/gpu)
        
        Returns
        -------
        encoder_inputs : torch.Tensor
            The constructed batches have the proper format to call
            step(...) later.
        decoder_inputs : torch.Tensor
            The constructed batches have the proper format to call
            step(...) later.
        target_weights : torch.Tensor
            The constructed batches have the proper format to call
            step(...) later.
        """

        input_size = self.input_size - self.len_z

        # Select entries at random
        all_keys = list(data.keys())
        chosen_keys = np.random.choice(len(all_keys), self.batch_size)

        # How many frames in total do we need?
        total_frames = self.source_seq_len + self.target_seq_len
        encoder_inputs = np.zeros(
            (self.batch_size, self.source_seq_len - 1, input_size),
            dtype=float)
        decoder_inputs = np.zeros(
            (self.batch_size, self.target_seq_len, input_size),
            dtype=float)
        decoder_outputs = np.zeros(
            (self.batch_size, self.target_seq_len, input_size),
            dtype=float)

        # Generate the sequences
        for i in range(self.batch_size):
            the_key = all_keys[chosen_keys[i]]

            # Get the number of frames
            n, _ = data[the_key].shape

            # Sample somewhere in the middle
            idx = np.random.randint(16, n - total_frames)

            # Select the data around the sampled points
            data_sel = data[the_key][idx:idx + total_frames, :]

            # Add the data
            encoder_inputs[i, :, 0:input_size] = data_sel[
                0:self.source_seq_len - 1, :]
            decoder_inputs[i, :, 0:input_size] = data_sel[
                self.source_seq_len - 1:self.source_seq_len +
                self.target_seq_len - 1, :]
            decoder_outputs[i, :,
                            0:input_size] = data_sel[self.source_seq_len:,
                                                          0:input_size]

        encoder_inputs = torch.tensor(encoder_inputs).float().to(device)
        decoder_inputs = torch.tensor(decoder_inputs).float().to(device)
        decoder_outputs = torch.tensor(decoder_outputs).float().to(device)

        return encoder_inputs, decoder_inputs, decoder_outputs

    def find_indices_srnn(self, data, action):
        """Find the same action indices as in SRNN.
        
        See https://github.com/asheshjain399/RNNexp/blob/master/structural_rnn/CRFProblems/H3.6m/processdata.py#L325

        Parameters
        ----------
        data:
            A list of sequences.
        action:
            The action.
        
        Returns
        -------
        idx : list
            A list of indices where the action is found.
        """

        # Used a fixed dummy seed, following
        # https://github.com/asheshjain399/RNNexp/blob/srnn/structural_rnn/forecastTrajectories.py#L29
        SEED = 1234567890
        rng = np.random.RandomState(SEED)

        subject = 5
        subaction1 = 1
        subaction2 = 2

        T1 = data[(subject, action, subaction1, 'even')].shape[0]
        T2 = data[(subject, action, subaction2, 'even')].shape[0]
        prefix, suffix = 50, 100

        # Test is performed always on subject 5
        # Select 8 random sub-sequences (by specifying their indices)
        idx = []
        idx.append(rng.randint(16, T1 - prefix - suffix))
        idx.append(rng.randint(16, T2 - prefix - suffix))
        idx.append(rng.randint(16, T1 - prefix - suffix))
        idx.append(rng.randint(16, T2 - prefix - suffix))
        idx.append(rng.randint(16, T1 - prefix - suffix))
        idx.append(rng.randint(16, T2 - prefix - suffix))
        idx.append(rng.randint(16, T1 - prefix - suffix))
        idx.append(rng.randint(16, T2 - prefix - suffix))

        return idx

    def get_batch_srnn(self, data, action, device):
        """Get a random batch of data from the specified bucket,
        prepare for step.

        Parameters
        ----------
        data: dict
            Dictionary with k:v, k=((subject, action, subsequence, 'even')),
            v=nxd matrix with a sequence of poses.
        action: str
            The action to load data from, e.g. 'walking'.
        
        Returns
        -------
        encoder_inputs : torch.Tensor
            The constructed batches have the proper format to call
            step(...) later.
        decoder_inputs : torch.Tensor
            The constructed batches have the proper format to call
            step(...) later.
        target_weights : torch.Tensor
            The constructed batches have the proper format to call
            step(...) later.
        """

        actions = [
            'directions', 'discussion', 'eating', 'greeting', 'phoning',
            'posing', 'purchases', 'sitting', 'sittingdown', 'smoking',
            'takingphoto', 'waiting', 'walking', 'walkingdog',
            'walkingtogether'
        ]

        if not action in actions:
            raise ValueError(f'Unrecognized action {action}')

        frames = {}
        frames[action] = self.find_indices_srnn(data, action)

        batch_size = 8  # we always evaluate 8 sequences
        subject = 5  # we always evaluate on subject 5
        source_seq_len = self.source_seq_len
        target_seq_len = self.target_seq_len
        input_size = self.input_size - self.len_z

        seeds = [(action, (i % 2) + 1, frames[action][i])
                 for i in range(batch_size)]

        encoder_inputs = np.zeros(
            (batch_size, source_seq_len - 1, input_size), dtype=float)
        decoder_inputs = np.zeros(
            (batch_size, target_seq_len, input_size), dtype=float)
        decoder_outputs = np.zeros(
            (batch_size, target_seq_len, input_size), dtype=float)

        # Compute the number of frames needed
        total_frames = source_seq_len + target_seq_len

        # Reproducing SRNN's sequence subsequence selection as done in
        # https://github.com/asheshjain399/RNNexp/blob/master/structural_rnn/CRFProblems/H3.6m/processdata.py#L343
        for i in range(batch_size):
            _, subsequence, idx = seeds[i]
            idx = idx + 50

            data_sel = data[(subject, action, subsequence, 'even')]
            data_sel = data_sel[(idx - source_seq_len):(idx +
                                                        target_seq_len), :]

            encoder_inputs[i, :, :] = data_sel[0:source_seq_len - 1, :]
            decoder_inputs[i, :, :] = data_sel[source_seq_len -
                                               1:(source_seq_len +
                                                  target_seq_len - 1), :]
            decoder_outputs[i, :, :] = data_sel[source_seq_len:, :]

        encoder_inputs = torch.tensor(encoder_inputs).float().to(device)
        decoder_inputs = torch.tensor(decoder_inputs).float().to(device)
        decoder_outputs = torch.tensor(decoder_outputs).float().to(device)

        return encoder_inputs, decoder_inputs, decoder_outputs


class MotionDiscriminator(nn.Module):
    """Sequence-to-sequence model for human motion prediction"""

    def __init__(self,
                 source_seq_len,
                 target_seq_len,
                 rnn_size,
                 batch_size,
                 learning_rate,
                 learning_rate_decay_factor,
                 number_of_actions,
                 dropout=0.3):
        """
        Parameters
        ----------
        source_seq_len: int
            The length of the input sequence.
        target_seq_len: int
            The length of the target sequence.
        rnn_size: int
            The number of units in the rnn.
        batch_size: int
            The size of the batches used during training; the model
            construction is independent of batch_size, so it can be
            changed after initialization if this is convenient, e.g.,
            for decoding.
        learning_rate: float
            The learning rate to start with.
        learning_rate_decay_factor: float
            The decay learning rate by this much when needed.
        number_of_actions: int
            The number of classes we have.
        """

        super(MotionDiscriminator, self).__init__()

        self.human_dofs = 54
        self.input_size = self.human_dofs + number_of_actions

        logging.info("Input size is {}".format(self.input_size))
        self.source_seq_len = source_seq_len
        self.target_seq_len = target_seq_len
        self.rnn_size = rnn_size
        self.batch_size = batch_size
        self.dropout = dropout

        # === Create the RNN that will summarizes the state ===
        self.cell = torch.nn.GRUCell(self.input_size, self.rnn_size)
        self.fc1 = nn.Linear(self.rnn_size, self.input_size)
        self.fc2 = nn.Linear(self.input_size * self.target_seq_len, 1)
        self.fc3 = nn.Sigmoid()

    def forward(self, generator_output, device):
        """Forward pass through the model.

        Parameters
        ----------
        encoder_inputs : torch.Tensor
            The constructed batches have the proper format to call
            step(...) later.
        decoder_inputs : torch.Tensor
            The constructed batches have the proper format to call
            step(...) later.
        device : torch.device
            The device to run the model on.

        Returns
        -------
        output_prob : torch.Tensor
            The output of the model.
        """

        def loop_function(prev, i):
            return prev

        batch_size = generator_output.shape[0]
        # To pass these data through a RNN we need to switch the first two dimensions
        generator_output = torch.transpose(generator_output, 0, 1)
        state = torch.zeros(batch_size, self.rnn_size).to(device)

        # Encoding
        for i in range(self.target_seq_len - 1):
            # Apply the RNN cell
            state = self.cell(generator_output[i], state)
            # Apply dropout in training
            state = F.dropout(state, self.dropout, training=self.training)

        outputs = []
        prev = None

        # Decoding, sequentially
        for i, inp in enumerate(generator_output):
            # Use teacher forcing?
            if prev is not None:
                inp = loop_function(prev, i)
            #inp = inp.detach()

            state = self.cell(inp, state)
            # Output is seen as a residual to the previous value
            output = inp + self.fc1(
                F.dropout(state, self.dropout, training=self.training))
            outputs.append(output.view([1, batch_size, self.input_size]))
            prev = output

        outputs = torch.cat(outputs, 0)
        outputs = torch.transpose(outputs, 0, 1)
        base = torch.flatten(outputs, start_dim=1)
        output_lineal = self.fc2(base)
        output_prob = self.fc3(output_lineal)

        return output_prob

You can define parameters by creating a dictionary:

In [ ]:
import os

from src.parsers import training_parser_from_dict


training_params = {
    'learning_rate': 0.0000001,
    'learning_rate_decay_factor': 0.95,
    'learning_rate_step': 10000,
    'batch_size': 8,
    'iterations': 2000, # Must be an integer
    'test_every': 100,
    'size': 512,
    'seq_length_in': 50,
    'seq_length_out': 10,
    'data_dir': os.path.normpath('./data/h3.6m/dataset'),
    'train_dir': os.path.normpath('./experiments/'),
    'action': 'walking',
    'log_file': '',
    'log_level': 20,
    'n_samples': 20
}

args = training_parser_from_dict(training_params)
args

### Model training

If you created a custom model, you may need to create a custom training funciton:

In [ ]:
"""Code for training an RNN for motion prediction."""

import logging
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim

IN_COLAB = 'google.colab' in sys.modules
if not IN_COLAB:
    from parsers import training_parser
    from utils.data_utils import read_all_data
    from utils.data_utils import define_actions
else:
    from src.utils.data_utils import read_all_data
    from src.utils.data_utils import define_actions


def train(args):
    """Train a seq2seq model on human motion.

    Parameters
    ----------
    args : argparse.Namespace
        Arguments from the parser.
    """

    # Set logger
    if args.log_file == '':
        logging.basicConfig(format='%(levelname)s: %(message)s',
                            level=args.log_level)
    else:
        logging.basicConfig(filename=args.log_file,
                            format='%(levelname)s: %(message)s',
                            level=args.log_level)

    # Set directory
    train_dir = os.path.normpath(
        os.path.join(args.train_dir, args.action, f'out_{args.seq_length_out}',
                     f'iterations_{args.iterations}', f'size_{args.size}',
                     f'lr_{args.learning_rate}'))

    # Detect device
    if torch.cuda.is_available():
        logging.info(torch.cuda.get_device_name(torch.cuda.current_device()))
    else:
        logging.info('cpu')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    logging.info('Train dir: ' + train_dir)
    os.makedirs(train_dir, exist_ok=True)

    # Set of actions
    actions = define_actions(args.action)
    number_of_actions = 1

    train_set, test_set, _, _, _, _ = read_all_data(actions,
                                                    args.seq_length_in,
                                                    args.seq_length_out,
                                                    args.data_dir)

    z = torch.rand((args.batch_size, args.seq_length_in - 1, 128)).to(device)

    # Create model for training only
    gen_model = MotionGenerator(
        args.seq_length_in,
        args.seq_length_out,
        args.n_samples,
        args.size,  # hidden layer size
        args.batch_size,
        args.learning_rate,
        args.learning_rate_decay_factor,
        number_of_actions)
    gen_model = gen_model.to(device)

    dis_model = MotionDiscriminator(
        args.seq_length_in,
        args.seq_length_out,
        args.size,  # hidden layer size
        args.batch_size,
        args.learning_rate,
        args.learning_rate_decay_factor,
        number_of_actions)
    dis_model = dis_model.to(device)

    # This is the training loop
    loss, val_loss = 0.0, 0.0
    current_step = 0
    all_losses = []
    all_val_losses = []

    criterion = nn.BCELoss()

    # The optimizer
    #optimiser = optim.SGD(model.parameters(), lr=args.learning_rate)
    gen_optimiser = optim.Adam(gen_model.parameters(),
                           lr=args.learning_rate,
                           betas=(0.9, 0.999))
    dis_optimiser = optim.Adam(dis_model.parameters(),
                           lr=args.learning_rate,
                           betas=(0.9, 0.999))

    for _ in range(args.iterations):
        # Set a flag to compute gradients
        gen_model.train()
        dis_model.train()


        # Forward pass (Discriminator)
        # Get batch from the training set
        encoder_inputs, decoder_inputs, decoder_outputs = gen_model.get_batch(
            train_set, actions, device)
        
        preds = gen_model(z, encoder_inputs, decoder_inputs, device)

        # Get min-error prediction
        errors = []
        for i in range(len(preds)):
            errors_i = (preds[i] - decoder_outputs)**2
            #errors_i = criterion(preds[i], decoder_outputs)
            errors_i = errors_i.mean()
            errors.append(errors_i)
    
        index = torch.argmin(torch.stack(errors))
        index = index.cpu().data.numpy()
        fake_outputs = preds[index]

        fake_gt_labels = torch.zeros(fake_outputs.size()[0]).to(device)
        real_gt_labels = torch.ones(fake_outputs.size()[0]).to(device)

        # === Training step (Discriminator) ===
        # encoder_inputs, decoder_inputs, device
        dis_model.zero_grad()
        fake_labels = dis_model(fake_outputs, device)
        real_labels = dis_model(decoder_outputs, device)

        fake_loss = (fake_gt_labels - fake_labels)**2
        #fake_loss = criterion(fake_gt_labels, fake_labels)
        real_loss = (real_gt_labels - real_labels)**2
        #real_loss = criterion(real_gt_labels, real_labels)

        d_loss = fake_loss + real_loss

        d_loss = d_loss.sum()
        d_loss.backward()
        dis_optimiser.step()

        # Forward pass (Generator)
        # Get batch from the training set
        encoder_inputs, decoder_inputs, decoder_outputs = gen_model.get_batch(
            train_set, actions, device)
        
        preds = gen_model(z, encoder_inputs, decoder_inputs, device)

        # Get min-error prediction
        errors = []
        for i in range(len(preds)):
            errors_i = (preds[i] - decoder_outputs)**2
            #errors_i = criterion(preds[i], decoder_outputs)
            errors_i = errors_i.mean()
            errors.append(errors_i)
    
        index = torch.argmin(torch.stack(errors))
        index = index.cpu().data.numpy()
        fake_outputs = preds[index]
        
        # === Training step (Generator) ===
        # Loss: Mean of mean Squared Errors for Discriminator
        gen_model.zero_grad()
        d_outputs = dis_model(fake_outputs, device)
        g_loss = (d_outputs - fake_gt_labels)**2
        #g_loss = criterion(d_outputs, fake_gt_labels)
        g_loss = g_loss.sum()
        g_loss.backward()
        gen_optimiser.step()
        step_loss = g_loss.cpu().data.numpy()

        loss += step_loss / args.test_every
        current_step += 1

        # === step decay ===
        if current_step % args.learning_rate_step == 0:
            args.learning_rate = args.learning_rate * args.learning_rate_decay_factor
            optimiser = optim.Adam(gen_model.parameters(),
                                   lr=args.learning_rate,
                                   betas=(0.9, 0.999))
            print(f'Decay learning rate. New value at {args.learning_rate}')

        # Once in a while, save checkpoint, print statistics.
        if current_step % args.test_every == 0:
            gen_model.eval()
            # === Validation ===
            encoder_inputs, decoder_inputs, decoder_outputs = gen_model.get_batch(
                test_set, actions, device)
            preds = gen_model(z, encoder_inputs, decoder_inputs, device)

            single_losses = []
        
            for i in range(len(preds)):
                step_loss_i = (preds[i] - decoder_outputs)**2
                #step_loss_i = criterion(preds[i], decoder_outputs)
                step_loss_i = step_loss_i.mean()
                single_losses.append(step_loss_i)
            
            step_loss = torch.mean(torch.stack(single_losses))
            val_loss = step_loss.sum()

            print('\n=================================\n'
                  f'Global step:         {current_step}\n'
                  f'Learning rate:       {args.learning_rate:.4}\n'
                  f'Train loss avg:      {loss:.4}\n'
                  '-------------------------------\n'
                  f'Val loss:            {val_loss:.4}\n'
                  '=================================\n')
            all_val_losses.append(
                [current_step, val_loss.cpu().detach().numpy()])
            all_losses.append([current_step, loss])
            torch.save(gen_model, train_dir + '/model_' + str(current_step))

            # Reset loss
            loss = 0

    vlosses = np.array(all_val_losses)
    tlosses = np.array(all_losses)

    # Plot losses
    plt.plot(vlosses[:, 0], vlosses[:, 1], 'b')
    plt.plot(tlosses[:, 0], tlosses[:, 1], 'r')
    plt.legend(['Validation loss', 'Training loss'])
    plt.show()


In [ ]:
train(args)

Once the model is trained, you can test it.

In [ ]:
from src.parsers import testing_parser_from_dict


testing_params = {
    'learning_rate': 0.0000001,
    'batch_size': 8,
    'iterations': 2000,
    'size': 512,
    'seq_length_out': 10,
    'horizon_test_step': 25,
    'data_dir': os.path.normpath('./data/h3.6m/dataset'),
    'train_dir': os.path.normpath('./experiments/'),
    'action': 'walking',
    'load_model': 2000,
    'log_level': 20,
    'log_file': '',
}

args = testing_parser_from_dict(testing_params)
args

In [ ]:
"""Code for training an RNN for motion prediction."""

import logging
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import h5py

IN_COLAB = 'google.colab' in sys.modules
if not IN_COLAB:
    from parsers import testing_parser
    from utils.data_utils import read_all_data
    from utils.data_utils import define_actions
    from utils.data_utils import rotmat_to_expmap
    from utils.data_utils import expmap_to_rotmat
    from utils.data_utils import unnormalize_data
    from utils.data_utils import revert_output_format
    from utils.evaluation import evaluate_batch
else:
    from src.utils.data_utils import read_all_data
    from src.utils.data_utils import define_actions
    from src.utils.data_utils import rotmat_to_expmap
    from src.utils.data_utils import expmap_to_rotmat
    from src.utils.data_utils import unnormalize_data
    from src.utils.data_utils import revert_output_format
    from src.utils.evaluation import evaluate_batch


def get_srnn_gts(actions,
                 model,
                 device,
                 test_set,
                 data_mean,
                 data_std,
                 dim_to_ignore,
                 to_euler=True):
    """Get the ground truths for srnn's sequences, and convert to Euler
    angles (the error is always computed in Euler angles).

    Parameters
    ----------
    actions : list
        A list of actions to get ground truths for.
    model: torch.nn.Module
        Training model we are using (we only use the "get_batch" method).
    device : torch.device
        Device to use for training.
    test_set: dict
        Dictionary with normalized training data.
    data_mean: np.array
        d-long vector with the mean of the training data.
    data_std: np.array
        d-long vector with the standard deviation of the training data.
    dim_to_ignore: np.array
        Dimensions that we are not using to train/predict.
    to_euler: bool
        Whether to convert the angles to Euler format or keep thm in
        exponential map.

    Returns
    -------
    srnn_gts_euler:
        A dictionary where the keys are actions, and the values are the
        ground_truth, denormalized expected outputs of srnns's seeds.
    """

    srnn_gts_euler = {}

    for action in actions:
        srnn_gt_euler = []
        # get_batch or get_batch_srnn
        _, _, srnn_expmap = model.get_batch_srnn(test_set, action, device)
        srnn_expmap = srnn_expmap.cpu()
        # expmap -> rotmat -> euler
        for i in np.arange(srnn_expmap.shape[0]):
            denormed = unnormalize_data(srnn_expmap[i, :, :], data_mean,
                                        data_std, dim_to_ignore, actions)
            if to_euler:
                for j in np.arange(denormed.shape[0]):
                    for k in np.arange(3, 97, 3):
                        denormed[j, k:k + 3] = rotmat_to_expmap(
                            expmap_to_rotmat(denormed[j, k:k + 3]))
            srnn_gt_euler.append(denormed)

        # Put back in the dictionary
        srnn_gts_euler[action] = srnn_gt_euler
    return srnn_gts_euler


def test(args):
    """Sample predictions for srnn's seeds.

    Parameters
    ----------
    args : argparse.Namespace
        Arguments from the parser.
    """

    # Set logger
    if args.log_file == '':
        logging.basicConfig(format='%(levelname)s: %(message)s',
                            level=args.log_level)
    else:
        logging.basicConfig(filename=args.log_file,
                            format='%(levelname)s: %(message)s',
                            level=args.log_level)

    # Set directory
    train_dir = os.path.normpath(
        os.path.join(args.train_dir, args.action, f'out_{args.seq_length_out}',
                     f'iterations_{args.iterations}', f'size_{args.size}',
                     f'lr_{args.learning_rate}'))

    # Detect device
    if torch.cuda.is_available():
        logging.info(torch.cuda.get_device_name(torch.cuda.current_device()))
    else:
        logging.info('cpu')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    logging.info('Train dir: ' + train_dir)
    os.makedirs(train_dir, exist_ok=True)

    # Set of actions
    actions = define_actions(args.action)
    nsamples = 8

    # Create the model
    logging.info(f'Creating a model with {args.size} units.')
    logging.info('Loading model')
    model = torch.load(train_dir + '/model_' + str(args.load_model))
    model.source_seq_len = 50
    model.target_seq_len = 100
    model = model.to(device)
    logging.info('Model created')

    # Load all the data
    _, test_set, data_mean, data_std, dim_to_ignore, _ = read_all_data(
        actions, 50, args.seq_length_out, args.data_dir)

    # === Read and denormalize the gt with srnn's seeds, as we'll need them
    # many times for evaluation in Euler Angles ===
    srnn_gts_expmap = get_srnn_gts(actions,
                                   model,
                                   device,
                                   test_set,
                                   data_mean,
                                   data_std,
                                   dim_to_ignore,
                                   to_euler=False)
    srnn_gts_euler = get_srnn_gts(actions, model, device, test_set, data_mean,
                                  data_std, dim_to_ignore)

    # Clean and create a new h5 file of samples
    SAMPLES_FNAME = 'samples.h5'
    try:
        os.remove(SAMPLES_FNAME)
    except OSError:
        pass

    # Predict and save for each action
    z = torch.rand((args.batch_size, 49, 128)).to(device)
    for action in actions:

        # Make prediction with srnn' seeds
        encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch_srnn(
            test_set, action, device)
        # Forward pass
        srnn_poses = model(z, encoder_inputs, decoder_inputs, device)

        srnn_losses = []
        for i in range(len(srnn_poses)):
            srnn_loss_i = (srnn_poses[i] - decoder_outputs)**2
            srnn_loss_i = srnn_loss_i.mean()
            srnn_losses.append(srnn_loss_i)
    
        index = torch.argmin(torch.stack(srnn_losses))
        index = index.cpu().data.numpy()

        srnn_poses = srnn_poses[index].cpu().data.numpy()
        srnn_poses = srnn_poses.transpose([1, 0, 2])


        # Restores the data in the same format as the original: dimension 99.
        # Returns a tensor of size (batch_size, seq_length, dim) output.
        srnn_pred_expmap = revert_output_format(srnn_poses, data_mean,
                                                data_std, dim_to_ignore,
                                                actions)

        # Save the samples
        with h5py.File(SAMPLES_FNAME, 'a') as hf:
            for i in np.arange(nsamples):
                # Save conditioning ground truth
                node_name = f'expmap/gt/{action}_{i}'
                hf.create_dataset(node_name, data=srnn_gts_expmap[action][i])
                # Save prediction
                node_name = f'expmap/preds/{action}_{i}'
                hf.create_dataset(node_name, data=srnn_pred_expmap[i])

        # Compute and save the errors here
        mean_errors_batch = evaluate_batch(srnn_pred_expmap,
                                           srnn_gts_euler[action])
        logging.info(
            'Mean error for test data along the horizon on action {}: {}'.
            format(action, mean_errors_batch))
        logging.info(
            'Mean error for test data at horizon {} on action {}: {}'.format(
                args.horizon_test_step, action,
                mean_errors_batch[args.horizon_test_step]))
        with h5py.File(SAMPLES_FNAME, 'a') as hf:
            node_name = f'mean_{action}_error'
            hf.create_dataset(node_name, data=mean_errors_batch)
    return

In [ ]:
test(args)

> If you need a custom function to test your model, you can create a new cell with a structure similar to the custom training section.

After testing the model, you can create an animation of the results. This will save all the output frames so we can later create a gif animation.

In [ ]:
from src.parsers import animation_parser_from_dict


animation_params = {
    'sample_id': 0,
    'imgs_dir': os.path.normpath('./images/')
}

args = animation_parser_from_dict(animation_params)
args

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from src.animate import animate



animate(args)

Let's create the gif animation.

In [ ]:
from src.animate import create_gif


create_gif('./images/', '.', filename='animation.gif')

Congrats, you're done! 🎉

---